# Exercise - VaR

## Data

This problem uses `weekly` return data from `data/spx_returns_weekly.xlsx`.

Choose any `4` stocks to evaluate below.

For example, 
* `AAPL`
* `META`
* `NVDA`
* `TSLA`

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

TICKS = ["AMD", "ADBE", "ORCL", "BKNG"]

stock_rets = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name="s&p500 rets").set_index("date")[TICKS]
bench_rets = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name="benchmark rets").set_index("date")

stock_rets.head()

,AMD,ADBE,ORCL,BKNG
date,,,,
2015-01-09,-0.022305,-0.006912,-0.018548,-0.078893
2015-01-16,-0.091255,-0.001531,0.002765,-0.041579
2015-01-23,0.025105,0.032483,0.015628,0.029527
2015-01-30,0.048980,-0.053065,-0.052048,-0.027466
2015-02-06,0.178988,0.034365,0.026021,0.012818


In [6]:
bench_rets.head()

,SPY,BTC,USO,TLT,IEF,IYR,GLD
date,,,,,,,
2015-01-09,-0.005744,-0.079179,-0.080945,0.029453,0.013517,0.029953,0.027875
2015-01-16,-0.012827,-0.281115,0.002735,0.016175,0.010188,0.019471,0.044858
2015-01-23,0.016565,0.137612,-0.072559,0.011863,0.001558,0.007958,0.013957
2015-01-30,-0.026931,-0.030969,0.048235,0.026044,0.011992,-0.013361,-0.006279
2015-02-06,0.030584,-0.027431,0.092593,-0.051020,-0.022724,-0.013173,-0.038963


# 1 Diversification

## 1.1

Using the full sample, calculate for each series the (unconditional) 
* volatility
* empirical VaR (.05)
* empirical CVaR (.05)

Recall that by **empirical** we refer to the direct quantile estimation. (For example, using `.quantile()` in pandas.

## 1.2
Form an equally-weighted portfolio of the investments.

Calculate the statistics of `1.1` for this portfolio, and compare the results to the individual return statistics. What do you find? What is driving this result?

## 1.3
Re-calculate `1.2`, but this time drop your most volatile asset, and replace the portion it was getting with 0. (You could imagine we're replacing the most volatile asset with a negligibly small risk-free rate.)

In comparing the answer here to 1.2, how much risk is your most volatile asset adding to the portfolio? Is this in line with the amount of risk we measured in the stand-alone risk-assessment of `1.1`?

***

# 2. Dynamic Measures

## 2.1 

Let's measure the **conditional** statistics of the equally-weighted portfolio of `1.2`, as of the end of the sample.

#### Volatility
For each security, calculate the **rolling** volatility series, $\sigma_t$, with a window of $m=26$.

The value at $\sigma_t$ in the notes denotes the estimate using data through time $t-1$, and thus (potentially) predicting the volatility at $\sigma_{t}$. 

#### Mean
Suppose we can approximate that the daily mean return is zero.

#### VaR
Calculate the **normal VaR** and **normal CVaR** for $q=.05$ and $\tau=1$ as of the end of the sample.Use the approximation, $\texttt{z}_{.05} = -1.65$.

#### Notation Note
In this setup, we are using a forecasted volatility, $\sigma_t$ to estimate the VaR return we would have estimated at the end of $t-1$ in prediction of time $t$.

#### Conclude and Compare
Report
* volatility (annualized).
* normal VaR (.05)
* normal CVaR (.05)

How do these compare to the answers in `1.2`?

## 2.2

Backtest the VaR using the **hit test**. Namely, check how many times the realized return at $t$ was smaller than the VaR return calculated using $\sigma_t$, (where again remember the notation in the notes uses $\sigma_t$ as a vol based on data through $t-1$.)

Report the percentage of "hits" using both the 
* expanding volatility
* rolling volatility

***